# Portafolios de Inversión
## Tarea 7

Presenta : 

- José Armando Melchor Soto


---
### Librerías 



In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize 

---

#### Indicaciones 

1.- Función de Métricas de Desempeño. (20%)

Genera una función en python que reciba un data frame con la evolución histórica de n cantidad de estrategías de inversión y calcule todas las métricas de desempeño vistas en las dos clases anteriores (Rendimiento, Volatilidad, Ratio de Sharpe, Downside, Upside, Omega, Beta, Alfa de Jensen, Ratio de Treynor, Ratio de Sortino).
 

2.- Backtesting de Portafolio. (50%)

Asume que quieres invertir en un portafolio compuesto por las siguientes acciones mexicanas: LIVEPOLC1.MX, KOFUBL.MX, CEMEXCPO.MX, CHDRAUIB.MX, CUERVO.MX, LABB.MX, RA.MX.
Debido a que no sabes como ponderarlos, decides realizar el backtesting para el portafolio para cuatro estrategias de Asset Allocation: Mínima Varianza, Máximo de Sharpe, Semivarianza Target y Omega.
Realiza el backtesting utilizando datos para simulación del 23-06-2024 hasta el 23-06-2025, toma un período anterior con el mismo horizonte temporal para optimizar las ponderaciones.
Recuerda siempre comparar con una estrategia de inversión pasiva, en este caso la opción lógica es el IPC México.
Gráfica la evolución histórica de las cinco alternativas de inversión.
Con los resultados del backtesting histórico calcula e interpreta las métricas de desempeño para las cinco alternativas de estrategias de inversión simuladas.
 

3.- Selección de Estrategia de Inversión. (20%)

Utilizando los resultados de tu backtesting y las métricas de desempeño, selecciona una estrategia de inversión. Realiza una conclusión clara, breve y robusta de porque es la mejor estrategia PARA TI, apoyate en las métricas obtenidas y en tu perfil de aversión al riesgo.
 

4.- Calculo de Pesos para Portafolio Real. (10%)

En base a la estrategia de Asset Allocation seleccionada en el punto anterior, calcula los pesos eficientes y la cantidad de acciones que utilizarías para invertir en la realidad en tu portafolio. Recuerda tomar la ventana de datos más reciente, en función del horizonte temporal tomado. 

---

#### Funciones 

##### Función Mínima Varianza 

In [6]:
def opt_min_var(rets):
    
    cov=rets.cov()
    n_assets = len(rets.columns)
    
    var = lambda w: w.T @ cov @ w
    
    w0=np.ones(n_assets)/n_assets
    
    bounds=[(0, 1)]*n_assets
    
    constraint=lambda w: sum(w)-1
    
    result=minimize(fun=var, x0=w0, bounds=bounds, constraints={'fun': constraint, 'type': 'eq'}, tol=1e-16)
    
    return result.x

##### Función Máximo de Sharpe

In [7]:
def opt_max_sharpe(rets, rf):
    
    E=rets.mean()
    cov=rets.cov()
    n_assets = len(rets.columns)
    
    rs = lambda w: -((w.T @ E - rf/252) / np.sqrt(w.T @ cov @ w))
    
    w0=np.ones(n_assets)/n_assets
    
    bounds=[(0, 1)]*n_assets
    
    constraint=lambda w: sum(w)-1
    
    result=minimize(fun=rs, x0=w0, bounds=bounds, constraints={'fun': constraint, 'type': 'eq'}, tol=1e-16)
    
    return result.x

##### Función Backtesting 

In [8]:
def static_backtesting(prices: pd.DataFrame, benchmark: pd.DataFrame, weights: dict, capital: float):
    
    # Obtener rendimientos de precios y benchmark
    rets_assets = prices.pct_change().dropna()
    rets_benchmark = benchmark.pct_change().dropna()
    
    # data frame vacio para llenar historia
    
    df = pd.DataFrame(index=rets_assets.index)
    
    # Obtener evolución para cada estrategia dada
    for strategy in weights.keys():
        # para benchmark
        if strategy == 'benchmark':
            temp = rets_benchmark + 1
            temp.iloc[0]=capital
            temp = temp.cumprod()
            df[strategy] = temp.values            
    
        # para otras estrategias
    
        else:
            temp = (rets_assets * weights.get(strategy)).sum(axis=1)+1
            temp.iloc[0]=capital
            temp = temp.cumprod()
            df[strategy] = temp.values    

    # Graficar la evolucion para ambos portafolios
    plt.figure(figsize=(12,6))
    plt.plot(df, label=df.keys())
    plt.title('Backtesting of Strategies')
    plt.xlabel('Date')
    plt.ylabel('Portafolio Value')
    plt.legend()
    plt.show()


    return df

##### Función Metricas de Desempeño

In [ ]:
def metricas_anuales(history, rf):
    # Returns
    rets= history.pct_change().dropna()
    
    # Matriz de Cov
    cov_hist = rets.cov()

    # Dataframe
    metricas = pd.DataFrame(index=rets.columns)

    # Rendimiento 
    metricas['Rendimiento Promedio Anual'] = R_p = rets.mean()*252*100
    
    # Volatilidad 
    metricas['Volatilidad Anual'] = vol_anual = rets.std()*np.sqrt(252)*100
    
    # Ratio de Sharpe
    metricas['Ratio de Sharpe'] = (R_p - rf) / vol_anual

    # Downside Risk
    metricas['Downside Risk'] = downside = rets[rets < 0].std() * np.sqrt(252) * 100

    # Upside Risk
    metricas['Upside Risk'] = upside = rets[rets > 0].std() * np.sqrt(252) * 100

    # Omega
    metricas['Omega'] =omega= upside / downside

    # Beta

    covs = cov_hist['Benchmark']
    varianza_market = covs['Benchmark']
    metricas['Beta'] = beta = covs / varianza_market

    # Alfa de Jensen

    R_m = rets['Benchmark']
    alpha_Jensen = R_p - (rf + beta * (R_m - rf))

    # Ratio de Treynor
    metricas['Ratio de Treynor'] = (R_p - rf) / beta

    # Ratio de Sortino
    metricas['Ratio de Sortino'] = (R_p - rf) / downside


---

#### Construcción del Portafolio 

##### Activos 

| **Empresa**                             | **Ticker**        | **Sector Económico**                          |
|----------------------------------------|-------------------|-----------------------------------------------|
| El Puerto de Liverpool, S.A.B. de C.V. | LIVEPOLC-1.MX     | Consumo                 |
| Coca-Cola FEMSA, S.A.B. de C.V.        | KOFUBL.MX         | Consumo Básico             |
| Cemex, S.A.B. de C.V.                  | CEMEXCPO.MX       | Materiales           |
| Grupo Comercial Chedraui               | CHDRAUIB.MX       | Consumo Básico          |
| Becle, S.A.B. de C.V. (José Cuervo)    | CUERVO.MX         | Consumo Básico       |
| Genomma Lab Internacional, S.A.B. de C.V. | LABB.MX         | Salud  |
| Rassini, S.A.B. de C.V.                | RA.MX             | Industriales                   |



##### Benchmark

##### Sector económico :   

- Vanguard Consumer Staples Index Fund ETF Shares (VDC)

---

#### Importación de los datos

##### Activos 

In [ ]:
prices = yf.download(['LIVEPOLC-1.MX', 'KOFUBL.MX', 'CEMEXCPO.MX', 'CHDRAUIB.MX', 'CUERVO.MX', 'LABB.MX', 'RA.MX'], start='2024-06-23', end='2025-06-23')['Close']
prices.head()

[*********************100%***********************]  7 of 7 completed


Ticker,CEMEXCPO.MX,CHDRAUIB.MX,CUERVO.MX,KOFUBL.MX,LABB.MX,LIVEPOLC-1.MX,RA.MX
Date,,,,,,,
2024-06-24,11.448030,126.384598,33.042759,150.527802,16.683031,124.302391,123.745781
2024-06-25,11.548014,125.912430,32.741837,153.082977,16.692781,124.655746,124.317032
2024-06-26,11.598005,124.151619,32.256485,152.003448,16.332016,128.159866,128.606079
2024-06-27,11.717984,124.515594,32.372967,154.984497,16.517273,129.396591,126.236794
2024-06-28,11.697988,124.722160,32.033222,155.846130,16.751284,129.053055,128.259598


##### Benchmarck 

In [ ]:
benchmark = yf.download('VDC', start='2024-06-23', end='2025-06-23')['Close']
benchmark.head()

[*********************100%***********************]  1 of 1 completed


Ticker,VDC
Date,
2020-01-02,140.328171
2020-01-03,140.082397
2020-01-06,140.293060
2020-01-07,139.222183
2020-01-08,139.775192


---